In [1]:
from replay.models import Wilson
from simulator.modules import Simulator, Generator, ResponseHeuristic
from simulator.utils import pandas_to_spark

import numpy as np

/data/home/anthony/.conda/envs/simulator/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import numpy as np

from simulator.utils import pandas_to_spark

users_df = pd.DataFrame(data=np.random.normal(0, 1, size=(1000, 35)), columns=[f'user_attr_{i}' for i in range(35)])
items_df = pd.DataFrame(data=np.random.normal(1, 1, size=(200, 20)), columns=[f'item_attr_{i}' for i in range(20)])
users_df['user_idx'] = np.arange(len(users_df))
items_df['item_idx'] = np.arange(len(items_df))
history_df = pd.DataFrame()
history_df['user_idx'] = np.random.randint(0, 1000, size=3000)
history_df['item_idx'] = np.random.randint(0, 200, size=3000)
history_df['relevance'] = np.random.randint(0, 2, size=3000)
history_df['timestamp'] = 0
history_df = history_df.drop_duplicates(subset=['user_idx', 'item_idx'], ignore_index=True)

In [3]:
gen = Generator.load('models/demo_wilson')

sim = Simulator(generator=gen)
sim.init(
    num_users=100,
    num_items=10,
    user_key_col='user_idx',
    item_key_col='item_idx',
    user_df=pandas_to_spark(users_df),
    item_df=pandas_to_spark(items_df),
    history_df=pandas_to_spark(history_df)
)

2022-05-27 09:27:35.981282: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-27 09:27:52.702876: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17677 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2022-05-27 09:27:52.707453: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10233 MB memory:  -> device: 1, name: A100-SXM4-40GB, pci bus id: 0000:0f:00.0, compute capability: 8.0
22/05/27 09:27:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using S

In [4]:
train_users = sim.sample_users(-1)

train_log, train_users, train_items = sim.get_train_log(train_users)

model = Wilson()
model.fit(
    log=train_log
)

05-27 09:28:08 I deeptables.m.deeptable.py 685 - Perform prediction...
05-27 09:28:08 I deeptables.m.preprocessor.py 242 - Transform [X]...
05-27 09:28:08 I deeptables.m.preprocessor.py 249 - transform_X taken 0.07505297660827637s
05-27 09:28:08 I deeptables.m.deepmodel.py 130 - Performing predictions...
05-27 09:28:08 I deeptables.u.dataset_generator.py 240 - create dataset generator with _TFDGForPandas, batch_size=128, shuffle=False, drop_remainder=False
05-27 09:28:10 I deeptables.m.deeptable.py 559 - predict_proba taken 1.934863805770874s
05-27 09:28:10 I deeptables.m.deeptable.py 594 - Reverse indicators to labels.


2022-05-27 09:28:10.712201: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
22/05/27 09:28:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 09:28:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 09:28:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 09:28:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 09:28:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/0

In [5]:
def zero_resp(a, b):
    return np.zeros(len(a))

r = ResponseHeuristic(heuristic_func=zero_resp)

for i in range(10):
    users = sim.sample_users(12)

    log, users, items = sim.get_user_items(users, 4)

    recs = model.predict(
        log=log,
        k=2,
        users=users.select('user_idx'),
        items=items.select('item_idx')
    )

    if recs.count() <= 0:
        continue

    true_resp = sim.sample_responses(
        recommendations_df=recs,
        alpha1=0.0,
        alpha2=1.0,
        alpha3=0.0,
        response_func=r,
        theta=0.8,
        save_history=True
    )

27-May-22 09:28:13, replay, WARNING: This model can't predict cold items, they will be ignored
27-May-22 09:28:13, replay, WARNING: This model can't predict cold items, they will be ignored
22/05/27 09:28:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 09:28:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 09:29:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 09:29:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


05-27 09:29:46 I deeptables.m.deeptable.py 685 - Perform prediction...
05-27 09:29:46 I deeptables.m.preprocessor.py 242 - Transform [X]...
05-27 09:29:46 I deeptables.m.preprocessor.py 249 - transform_X taken 0.07633185386657715s
05-27 09:29:46 I deeptables.m.deepmodel.py 130 - Performing predictions...
05-27 09:29:46 I deeptables.u.dataset_generator.py 240 - create dataset generator with _TFDGForPandas, batch_size=128, shuffle=False, drop_remainder=False


05-27 09:29:46 I deeptables.m.deeptable.py 559 - predict_proba taken 0.17595958709716797s
05-27 09:29:46 I deeptables.m.deeptable.py 594 - Reverse indicators to labels.


27-May-22 09:29:47, replay, WARNING: This model can't predict cold items, they will be ignored
27-May-22 09:29:47, replay, WARNING: This model can't predict cold items, they will be ignored
22/05/27 09:29:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 09:29:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 09:29:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 09:30:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 09:30:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performanc

05-27 09:33:28 I deeptables.m.deeptable.py 685 - Perform prediction...
05-27 09:33:28 I deeptables.m.preprocessor.py 242 - Transform [X]...
05-27 09:33:28 I deeptables.m.preprocessor.py 249 - transform_X taken 0.07209920883178711s
05-27 09:33:28 I deeptables.m.deepmodel.py 130 - Performing predictions...
05-27 09:33:28 I deeptables.u.dataset_generator.py 240 - create dataset generator with _TFDGForPandas, batch_size=128, shuffle=False, drop_remainder=False
05-27 09:33:28 I deeptables.m.deeptable.py 559 - predict_proba taken 0.17087721824645996s
05-27 09:33:28 I deeptables.m.deeptable.py 594 - Reverse indicators to labels.


...

05-27 09:50:10 I deeptables.m.deeptable.py 685 - Perform prediction...
05-27 09:50:10 I deeptables.m.preprocessor.py 242 - Transform [X]...
05-27 09:50:10 I deeptables.m.preprocessor.py 249 - transform_X taken 0.07230973243713379s
05-27 09:50:10 I deeptables.m.deepmodel.py 130 - Performing predictions...
05-27 09:50:10 I deeptables.u.dataset_generator.py 240 - create dataset generator with _TFDGForPandas, batch_size=128, shuffle=False, drop_remainder=False
05-27 09:50:11 I deeptables.m.deeptable.py 559 - predict_proba taken 0.18144512176513672s
05-27 09:50:11 I deeptables.m.deeptable.py 594 - Reverse indicators to labels.


27-May-22 09:50:13, replay, WARNING: This model can't predict cold items, they will be ignored
27-May-22 09:50:13, replay, WARNING: This model can't predict cold items, they will be ignored
22/05/27 09:50:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 09:50:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 09:50:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 09:50:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 09:50:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performanc

05-27 10:05:59 I deeptables.m.deeptable.py 685 - Perform prediction...
05-27 10:05:59 I deeptables.m.preprocessor.py 242 - Transform [X]...
05-27 10:05:59 I deeptables.m.preprocessor.py 249 - transform_X taken 0.07465696334838867s
05-27 10:05:59 I deeptables.m.deepmodel.py 130 - Performing predictions...
05-27 10:05:59 I deeptables.u.dataset_generator.py 240 - create dataset generator with _TFDGForPandas, batch_size=128, shuffle=False, drop_remainder=False


05-27 10:06:00 I deeptables.m.deeptable.py 559 - predict_proba taken 0.20258116722106934s
05-27 10:06:00 I deeptables.m.deeptable.py 594 - Reverse indicators to labels.


27-May-22 10:06:06, replay, WARNING: This model can't predict cold items, they will be ignored
27-May-22 10:06:06, replay, WARNING: This model can't predict cold items, they will be ignored
22/05/27 10:06:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 10:06:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 10:06:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 10:06:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/27 10:06:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performanc

In [ ]:
sim.history_df.toPandas()

In [ ]:
true_resp.toPandas()

In [ ]:
recs.show()